In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns

from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
from sklearn.model_selection import GroupKFold

plt.style.use("fivethirtyeight")
mypal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [5]:
!ls -GFlash ../input/march-machine-learning-mania-2024/

total 144M

   0 drwxr-xr-x 2 nobody    0 May 13 11:16 ./

4.0K drwxr-xr-x 3 root   4.0K Jun  5 15:37 ../

4.0K -rw-r--r-- 1 nobody 1.4K May 13 11:16 2024_tourney_seeds.csv

 12K -rw-r--r-- 1 nobody 9.1K May 13 11:16 Cities.csv

4.0K -rw-r--r-- 1 nobody 1.7K May 13 11:16 Conferences.csv

168K -rw-r--r-- 1 nobody 168K May 13 11:16 MConferenceTourneyGames.csv

2.5M -rw-r--r-- 1 nobody 2.5M May 13 11:16 MGameCities.csv

111M -rw-r--r-- 1 nobody 111M May 13 11:16 MMasseyOrdinals_thruSeason2024_day128.csv

 72K -rw-r--r-- 1 nobody  72K May 13 11:16 MNCAATourneyCompactResults.csv

132K -rw-r--r-- 1 nobody 129K May 13 11:16 MNCAATourneyDetailedResults.csv

 16K -rw-r--r-- 1 nobody  16K May 13 11:16 MNCAATourneySeedRoundSlots.csv

 40K -rw-r--r-- 1 nobody  38K May 13 11:16 MNCAATourneySeeds.csv

 52K -rw-r--r-- 1 nobody  50K May 13 11:16 MNCAATourneySlots.csv

5.3M -rw-r--r-- 1 nobody 5.3M May 13 11:16 MRegularSeasonCompactResults.csv

 11M -rw-r--r-- 1 nobody  11M May 13 11:16 MRegularSeasonD

# **Files we are interested in:**

* ***MRegularSeasonCompactResults.csv & WMRegularSeasonCompactResults.csv**

    All game results from the regular season.
* **MNCAATourneyCompactResults.csv & WNCAATourneyCompactResults.csv**

    All game results from past tournaments.
* **MNCAATourneySeeds.csv & MNCAATourneySeeds.csv**

     The seeding for the tournaments
* *** 2024_tourney_seeds.csv**

   File that will be updated with 2024 seeds once released (2023 seeds prior to that)

In [6]:
DATA_PATH = "../input/march-machine-learning-mania-2024/"

In [7]:
df_seeds = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv").assign(League="W"),
    ],
).reset_index(drop=True)

df_season_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)

df_tourney_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneyCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)

# **Creating Team Season Results**

* We the the data from the existing format with 1 row per game
* New format has 1 row for each team's game - win or loss.
* This data can be aggregated for season metrics

In [10]:
df_team_season_results = pd.concat(
    [
        df_season_results[["Season", "League", "WTeamID", "DayNum", "WScore", "LScore"]]
        .assign(GameResult="W")
        .rename(
            columns={"WTeamID": "TeamID", "WScore": "TeamScore", "LScore": "OppScore"}
        ),
        df_season_results[["Season", "League", "LTeamID", "DayNum", "WScore", "LScore"]]
        .assign(GameResult="L")
        .rename(
            columns={"LTeamID": "TeamID", "LScore": "TeamScore", "WScore": "OppScore"}
        ),
    ]
).reset_index(drop=True)

# Create Season Features

* add some features to this data like the score differential

In [11]:
# Score Differential
df_team_season_results["ScoreDiff"] = (
    df_team_season_results["TeamScore"] - df_team_season_results["OppScore"]
)
df_team_season_results["Win"] = (df_team_season_results["GameResult"] == "W").astype(
    "int"
)

In [12]:
df_team_season_results.sample(10, random_state=529)

,Season,League,TeamID,DayNum,TeamScore,OppScore,GameResult,ScoreDiff,Win
493232,2022,M,1444,96,64,77,L,-13,0
71811,2002,M,1281,79,74,50,W,24,1
555548,2008,W,3276,115,67,69,L,-2,0
84226,2005,M,1393,40,86,56,W,30,1
129439,2014,M,1368,8,63,62,W,1,1
364900,1996,M,1148,85,46,116,L,-70,0
587485,2015,W,3394,18,66,68,L,-2,0
397470,2003,M,1266,129,76,83,L,-7,0
99374,2008,M,1177,47,93,88,W,5,1
532105,2003,W,3414,130,50,83,L,-33,0


# Aggregate for team's total season stats



In [13]:
# Aggregate the data
team_season_agg = (
    df_team_season_results.groupby(["Season", "TeamID", "League"])
    .agg(
        AvgScoreDiff=("ScoreDiff", "mean"),
        MedianScoreDiff=("ScoreDiff", "median"),
        MinScoreDiff=("ScoreDiff", "min"),
        MaxScoreDiff=("ScoreDiff", "max"),
        Wins=("Win", "sum"),
        Losses=("GameResult", lambda x: (x == "L").sum()),
        WinPercentage=("Win", "mean"),
    )
    .reset_index()
)

In [14]:
team_season_agg.head()

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,Wins,Losses,WinPercentage
0,1985,1102,M,-5.791667,-5.5,-41,29,5,19,0.208333
1,1985,1103,M,-3.043478,-2.0,-22,16,9,14,0.391304
2,1985,1104,M,7.800000,6.5,-12,25,21,9,0.700000
3,1985,1106,M,-3.791667,-1.5,-35,28,10,14,0.416667
4,1985,1108,M,7.960000,4.0,-15,35,19,6,0.760000


In [15]:
df_seeds["ChalkSeed"] = (
    df_seeds["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)

team_season_agg = team_season_agg.merge(
    df_seeds, on=["Season", "TeamID", "League"], how="left"
)

In [17]:
team_season_agg.shape, df_seeds.shape

((22150, 12), (4234, 5))

# Tournament Results Aggregation

In [18]:
df_team_tourney_results = pd.concat(
    [
        df_tourney_results[
            ["Season", "League", "WTeamID", "LTeamID", "WScore", "LScore"]
        ]
        .assign(GameResult="W")
        .rename(
            columns={
                "WTeamID": "TeamID",
                "LTeamID": "OppTeamID",
                "WScore": "TeamScore",
                "LScore": "OppScore",
            }
        ),
        df_tourney_results[
            ["Season", "League", "LTeamID", "WTeamID", "LScore", "WScore"]
        ]
        .assign(GameResult="L")
        .rename(
            columns={
                "LTeamID": "TeamID",
                "WTeamID": "OppTeamID",
                "LScore": "TeamScore",
                "WScore": "OppScore",
            }
        ),
    ]
).reset_index(drop=True)

df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype(
    "int"
)

In [19]:
df_team_tourney_results.head()

,Season,League,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win
0,1985,M,1116,1234,63,54,W,1
1,1985,M,1120,1345,59,58,W,1
2,1985,M,1207,1250,68,43,W,1
3,1985,M,1229,1425,58,55,W,1
4,1985,M,1242,1325,49,38,W,1


# Tourney Dataset with Features

* merge our team's regular season features with our tourney dataframe.
* This gives us the data format that we will use to train our model.
* target column is the "Winner" and the features are the regular season stats.

In [20]:
df_historic_tourney_features = df_team_tourney_results.merge(
    team_season_agg[
        ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed"]
    ],
    on=["Season", "League", "TeamID"],
    how="left",
).merge(
    team_season_agg[
        ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed"]
    ].rename(
        columns={
            "TeamID": "OppTeamID",
            "WinPercentage": "OppWinPercentage",
            "MedianScoreDiff": "OppMedianScoreDiff",
            "ChalkSeed": "OppChalkSeed",
        }
    ),
    on=["Season", "League", "OppTeamID"],
)

In [21]:
df_historic_tourney_features.head()

,Season,League,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win,WinPercentage,MedianScoreDiff,ChalkSeed,OppWinPercentage,OppMedianScoreDiff,OppChalkSeed
0,1985,M,1116,1234,63,54,W,1,0.636364,5.0,9.0,0.666667,9.5,8.0
1,1985,M,1120,1345,59,58,W,1,0.620690,2.0,11.0,0.680000,9.0,6.0
2,1985,M,1207,1250,68,43,W,1,0.925926,14.0,1.0,0.379310,-3.0,16.0
3,1985,M,1229,1425,58,55,W,1,0.740741,6.0,9.0,0.678571,2.5,8.0
4,1985,M,1242,1325,49,38,W,1,0.766667,5.5,3.0,0.740741,6.0,14.0
